# Flirt prediction model

In [ ]:
input
text msg 
filrt words , messages 


In [1]:
import pandas as pd
df = pd.read_csv('flirt_dataset.csv')
df.head()

,text,label
0,Good morning beautiful 😘,1
1,You looked amazing today 😍,1
2,I miss talking to you ❤️,1
3,You're really special to me 😉,1
4,I can't stop thinking about you 😘,1


In [2]:
import nltk

#step1 tokenize
from nltk.tokenize import sent_tokenize, word_tokenize

#step 2 stopwords
from nltk.corpus import stopwords

#step 3 stemming and lemma

from nltk.stem import PorterStemmer, WordNetLemmatizer

#step 4 regex
import re

#pip install emoji 
# step 5
import emoji

# step 6
from nltk import pos_tag

#step 7
from nltk import ne_chunk



In [ ]:

def download_packages(): 
    #downloading the required packages for processing
    nltk.download("punkt")
    nltk.download("punkt_tab")

    #stop words
    nltk.download("stopwords")
    # stemming and lemmetization
    nltk.download("wordnet")

    #POS and tagging 
    #POS
    # nltk.download('averaged_perceptron_tagger')
    nltk.download('averaged_perceptron_tagger_eng')
    # tagging
    nltk.download('tagsets_json')


    # named entity recognition package
    nltk.download('maxent_ne_chunker_tab')
    # nltk.download('maxent_ne_chunker')
    nltk.download('words')

# download_packages()

# packages_names = []
# for package in packages_names:
#     nltk.download(package)

In [3]:
def clean_text(text):
    import emoji, re

    # Convert emojis → words
    text = emoji.demojize(text)

    # Replace ":" so emoji names become single tokens
    text = text.replace(":", " ")

    # Lowercase
    text = text.lower()

    # Remove URLs
    text = re.sub(r"http\S+|www\.\S+", "", text)

    # Keep alphabets + underscore only
    text = re.sub(r"[^a-zA-Z_\s]", " ", text)

    # Remove extra spaces
    text = re.sub(r"\s+", " ", text).strip()

    return text


def text_preprocessing(df):    
    
    df['cleaned_text'] = df['text'].apply(lambda x: clean_text(x))
    return df

    # stopping_words = set(stopwords.words("english"))
    # filtered_words = [word for word in words_tokens if word not in stopping_words]
    # return clean_text

final_df = text_preprocessing(df)
# final_df.to_csv('flirt_cleaned_dataset.csv', index=False)


In [4]:
final_df

,text,label,cleaned_text
0,Good morning beautiful 😘,1,good morning beautiful face_blowing_a_kiss
1,You looked amazing today 😍,1,you looked amazing today smiling_face_with_hea...
2,I miss talking to you ❤️,1,i miss talking to you red_heart
3,You're really special to me 😉,1,you re really special to me winking_face
4,I can't stop thinking about you 😘,1,i can t stop thinking about you face_blowing_a...
...,...,...,...
187,Let me know,0,let me know
188,Please finish the work,0,please finish the work
189,Send me the link,0,send me the link
190,Any updates?,0,any updates


In [5]:
def text_representation(df):
    # Bag of Words
    from sklearn.feature_extraction.text import CountVectorizer
    vectorizer = CountVectorizer()
    X_bow = vectorizer.fit_transform(df['cleaned_text'])
    print("Bag of Words representation shape:", X_bow.shape)

    # TF-IDF
    from sklearn.feature_extraction.text import TfidfVectorizer
    tfidf_vectorizer = TfidfVectorizer()
    X_tfidf = tfidf_vectorizer.fit_transform(df['cleaned_text'])
    print("TF-IDF representation shape:", X_tfidf.shape)

text_representation(final_df)

Bag of Words representation shape: (192, 278)
TF-IDF representation shape: (192, 278)


In [6]:
final_df['cleaned_text']

0             good morning beautiful face_blowing_a_kiss
1      you looked amazing today smiling_face_with_hea...
2                        i miss talking to you red_heart
3               you re really special to me winking_face
4      i can t stop thinking about you face_blowing_a...
                             ...                        
187                                          let me know
188                               please finish the work
189                                     send me the link
190                                          any updates
191                                 working on the issue
Name: cleaned_text, Length: 192, dtype: object

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    final_df['cleaned_text'], final_df['label'], test_size=0.2, random_state=42
)
# from sklearn.feature_extraction.text import CountVectorizer
# cv = CountVectorizer()
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1,3), min_df=1, token_pattern=r"[A-Za-z_]+",sublinear_tf=True)
X_train_cv = tfidf_vectorizer.fit_transform(X_train)
X_test_cv = tfidf_vectorizer.transform(X_test)


In [8]:
X_test_cv[0]

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 9 stored elements and shape (1, 1157)>

In [9]:
# from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
model = LinearSVC()
model.fit(X_train_cv, y_train)


,penalty,'l2'
,loss,'squared_hinge'
,dual,'auto'
,tol,0.0001
,C,1.0
,multi_class,'ovr'
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,verbose,0
,random_state,None


In [10]:
pred = model.predict(X_test_cv)

print("Accuracy:", accuracy_score(y_test, pred))
print(classification_report(y_test, pred))


Accuracy: 0.9487179487179487
              precision    recall  f1-score   support

           0       1.00      0.90      0.95        21
           1       0.90      1.00      0.95        18

    accuracy                           0.95        39
   macro avg       0.95      0.95      0.95        39
weighted avg       0.95      0.95      0.95        39



In [11]:
type(clean_text)

function

In [13]:
sample = ["I miss you so much 😘", "Send me the excel file", "Call me when you are free","You’re trouble"]
sample_clean = [clean_text(s) for s in sample]
sample_vec = tfidf_vectorizer.transform(sample_clean)

result = model.predict(sample_vec)

labels = {0: 'not_flirt', 1: 'flirt'}
for each in zip(sample,result):
    print(each[0], ":",labels[each[1]])


I miss you so much 😘 : flirt
Send me the excel file : not_flirt
Call me when you are free : not_flirt
You’re trouble : flirt


#testing with our whatsapp dataset


In [14]:
import pandas as pd
df_whatsapp = pd.read_csv("whatsappmsg.txt", header=None,on_bad_lines='skip', encoding='utf-8')
df_whatsapp.head()

,0,1
0,8:01 am,1/10/2025 - Alice: Morning sunshine ☀️
1,8:02 am,1/10/2025 - Leela: Morning 😴 coffee first pls
2,8:10 am,1/10/2025 - Leela: What’s your plan today?
3,8:11 am,1/10/2025 - Alice: Just work calls 😩 you?
4,8:12 am,1/10/2025 - Leela: WFH today 🙌


In [15]:
df_whatsapp.columns=['Time', 'message']

In [16]:
Date = df_whatsapp['message'].str.split('-', n=1, expand=True) #stops with first split itself dont split if you say further hypens - 
Date

,0,1
0,1/10/2025,Alice: Morning sunshine ☀️
1,1/10/2025,Leela: Morning 😴 coffee first pls
2,1/10/2025,Leela: What’s your plan today?
3,1/10/2025,Alice: Just work calls 😩 you?
4,1/10/2025,Leela: WFH today 🙌
...,...,...
142,25/10/2025,Leela: Can’t sleep 😭
143,25/10/2025,Alice: Same d scrolling reels 🤦‍♀️
144,25/10/2025,Leela: 😂 we need a life
145,25/10/2025,Alice: Tomorrow gym 😤


In [17]:
Name = Date[1].str.split(':', n=1, expand=True) #stops with first split itself dont split if you say further hypens - 
Name

,0,1
0,Alice,Morning sunshine ☀️
1,Leela,Morning 😴 coffee first pls
2,Leela,What’s your plan today?
3,Alice,Just work calls 😩 you?
4,Leela,WFH today 🙌
...,...,...
142,Leela,Can’t sleep 😭
143,Alice,Same d scrolling reels 🤦‍♀️
144,Leela,😂 we need a life
145,Alice,Tomorrow gym 😤


In [18]:
df_whatsapp['Date'] = Date[0].str.strip()
df_whatsapp['Name'] = Name[0].str.strip()
df_whatsapp['Text'] = Name[1].str.strip()

In [19]:
df_whatsapp

,Time,message,Date,Name,Text
0,8:01 am,1/10/2025 - Alice: Morning sunshine ☀️,1/10/2025,Alice,Morning sunshine ☀️
1,8:02 am,1/10/2025 - Leela: Morning 😴 coffee first pls,1/10/2025,Leela,Morning 😴 coffee first pls
2,8:10 am,1/10/2025 - Leela: What’s your plan today?,1/10/2025,Leela,What’s your plan today?
3,8:11 am,1/10/2025 - Alice: Just work calls 😩 you?,1/10/2025,Alice,Just work calls 😩 you?
4,8:12 am,1/10/2025 - Leela: WFH today 🙌,1/10/2025,Leela,WFH today 🙌
...,...,...,...,...,...
142,9:31 pm,25/10/2025 - Leela: Can’t sleep 😭,25/10/2025,Leela,Can’t sleep 😭
143,9:32 pm,25/10/2025 - Alice: Same d scrolling reels 🤦‍♀️,25/10/2025,Alice,Same d scrolling reels 🤦‍♀️
144,9:33 pm,25/10/2025 - Leela: 😂 we need a life,25/10/2025,Leela,😂 we need a life
145,9:34 pm,25/10/2025 - Alice: Tomorrow gym 😤,25/10/2025,Alice,Tomorrow gym 😤


In [20]:
df_whatsapp.drop(columns=['message'], inplace=True)

In [21]:
df_whatsapp

,Time,Date,Name,Text
0,8:01 am,1/10/2025,Alice,Morning sunshine ☀️
1,8:02 am,1/10/2025,Leela,Morning 😴 coffee first pls
2,8:10 am,1/10/2025,Leela,What’s your plan today?
3,8:11 am,1/10/2025,Alice,Just work calls 😩 you?
4,8:12 am,1/10/2025,Leela,WFH today 🙌
...,...,...,...,...
142,9:31 pm,25/10/2025,Leela,Can’t sleep 😭
143,9:32 pm,25/10/2025,Alice,Same d scrolling reels 🤦‍♀️
144,9:33 pm,25/10/2025,Leela,😂 we need a life
145,9:34 pm,25/10/2025,Alice,Tomorrow gym 😤


In [22]:
#text_preprocessing
df_whatsapp['Cleaned_Text'] = df_whatsapp['Text'].apply(lambda x: clean_text(x))
df_whatsapp.head()

,Time,Date,Name,Text,Cleaned_Text
0,8:01 am,1/10/2025,Alice,Morning sunshine ☀️,morning sunshine sun
1,8:02 am,1/10/2025,Leela,Morning 😴 coffee first pls,morning sleeping_face coffee first pls
2,8:10 am,1/10/2025,Leela,What’s your plan today?,what s your plan today
3,8:11 am,1/10/2025,Alice,Just work calls 😩 you?,just work calls weary_face you
4,8:12 am,1/10/2025,Leela,WFH today 🙌,wfh today raising_hands


In [23]:
text_to_vectors = tfidf_vectorizer.transform(df_whatsapp['Cleaned_Text'])

results = model.predict(text_to_vectors)

labels = {0: 'not_flirt', 1: 'flirt'}
for each in zip(df_whatsapp['Cleaned_Text'],results):
    print(each[0], ":",labels[each[1]])

morning sunshine sun : flirt
morning sleeping_face coffee first pls : flirt
what s your plan today : not_flirt
just work calls weary_face you : flirt
wfh today raising_hands : not_flirt
did you finish that report : not_flirt
nope loudly_crying_face still halfway : not_flirt
lol same here face_with_tears_of_joy : not_flirt
procrastination buddies smiling_face_with_sunglasses : not_flirt
lunch curry_rice : not_flirt
ordering pizza face_savoring_food : not_flirt
save me a slice face_with_tears_of_joy : not_flirt
only if you bring dessert smirking_face : flirt
traffic is insane weary_face : not_flirt
i just want to teleport home melting_face : flirt
same dream face_with_tears_of_joy : not_flirt
your outfit yesterday was cute dress : flirt
aww thanks smiling_face_with_heart eyes it was a last minute pick : not_flirt
looked like a pro choice winking_face : flirt
need a new series recommendation loudly_crying_face : not_flirt
watch only murders in the building so good fire : not_flirt
adding 